## AES Function

In [1]:
#BELOW IS THE AES CODE

#!/usr/bin/env python
# coding: utf-8

# In[1]:


#import libraries
import numpy as np


# In[2]:


'''
Function Name: Matrix_To_String
  Convert input matrix to string
Parameters:
  input matrix
Returns:
  String containing the elements of the input matrix
'''
def Matrix_To_String(matrix):
    #gets size of matrix
    R = len(matrix)
    C = len(matrix[0])

    #iniztilize empty string
    string = '' 
    
    #iterate over the rows and the coloumns
    for j in range(C):
        for i in range(R):
            #add every element of the matrix to the string
            string += matrix[i][j] 
    
    #return string
    return string


# In[3]:


'''
Function Name: Convert_to_Matrix
  Convert input string or list to matrix of size 4 x 4
Parameters:
  input string or list
Returns:
  matrix
'''

def Convert_to_Matrix(data):
    #matrix has to be 4 x 4 
    R,C = 4,4
    #initialize numpy matrix
    matrix = np.zeros((R,C) , dtype = np.chararray)
    position = 0
    
    #iterate over the rows and coloums
    for j in range(C):
        for i in range(R):
            if(type(data) is str): #if string
                matrix[i][j] = data[position:position+2] #at position i and j, add the intended value to the matrix
                position += 2
                
            elif(type(data) is list): #if list
                matrix[i][j] = data[position] + data[position+1]
                position += 2
            
    return matrix


# In[4]:


'''
Function Name: Convert_to_Matrix_256
  Convert input string or list to matrix of size 4 x 8 (used for AES 256)
Parameters:
  input string or list
Returns:
  matrix
'''

def Convert_to_Matrix_256(data):
    R,C = 4,8 #matrix 4x8
    matrix = np.zeros((R,C) , dtype = np.chararray)
    position = 0
    
    #iterate over the rows and coloums
    for j in range(C):
        for i in range(R):
            if(type(data) is str): #if string
                matrix[i][j] = data[position:position+2]  #at position i and j, add the intended value to the matrix
                position += 2
                
            elif(type(data) is list): #if list
                matrix[i][j] = data[position] + data[position+1]
                position += 2
            
    return matrix


# In[5]:


'''
Function Name: HEX_Column_To_String
  convert a hexadecimal column in a given matrix to a string
Parameters:
  matrix column
Returns:
  string
'''

def HEX_Column_To_String(Column):
    string=Column[0]+Column[1]+Column[2]+Column[3]#add every element together in a list
    return string


# In[6]:


'''
Function Name: BIN_To_HEX
  Convert input binary number to hexadecimal number
Parameters:
  binary number
Returns:
  converted hexadecimal number
'''

def BIN_To_HEX(number):
    #use "hex" to convert the number to hexadecimal
    hex_output = hex(int(number, 2)) 
    #remove the 0x
    output=hex_output.replace('0x' , '')
    return output


# In[7]:


'''
Function Name: HEX_To_BIN
  Convert input hexadecimal number to  binary number
Parameters:
  hexadecimal number (should be 16 length and if not pad or fill it with zeros)
Returns:
  converted binary number
'''

def HEX_To_BIN(data):
    #use "bin" to convert the number to binary
    bin_output = bin(int(data , 16))
    #pad the string with zeros
    bin_pad = bin_output[2:].zfill(8)
    return bin_pad


# In[8]:


#Function to add a zero to a given string
#takes input a string and add zero to the starting of the string
#output string

def add_zero(string):
    for i in range(8-len(string)): 
        string = '0'+ string
        
    return string


# In[9]:


#shift left first two indecies in a given word (used in key expansion)
#takes input word string
#output string

def Left_Shift(word):
    output=word[2:len(word)] + word[0:2]
    return output


# In[10]:


def Key_Expand_256(key):
    k = Convert_to_Matrix_256(key) #2hex per cell
    w = list()  #word is column major
    round_number = 0
    RC = ['01' , '02' , '04' , '08' , '10' , '20' , '40' , '80' , '1b' , '36', '01' , '02','04' , '08']
    
    for j in range(8):
        word = HEX_Column_To_String(k[:,j])
        w.append(word)

    for i in range(8 , 60, 1):
        word = ''
        if(i % 8 == 0): 
            output= RC[round_number]+'000000'
            output1=Left_Shift(w[(i-1)])
            output2=Sub_Byte(output1 , 'E')
            word = HEX_XOR(w[(i-8)] , HEX_XOR(output2 ,output))
            round_number += 1
        elif (i%4 == 0):
            outputt = Sub_Byte(w[i-1] , 'E')
            word = HEX_XOR(w[(i-8)] , outputt)

        else:
            word = HEX_XOR(w[(i-8)] , w[(i-1)])
            
        w.append(word)            
    return w


# In[12]:


#The rotate function is used to rotate rows n number of times
#takes input rows and n
#return modified rows
def rotate(row, n):
  return row[n:] + row[:n]


# In[13]:


#Galois Field Multiplication function used to multiply matrices.

def GF_Mul(S , M):
    result = ''
    S = HEX_To_BIN(S) #first convert the number to binary

    #if we multiply by "01" it's just the same result
    if(M == '01'): 
        result = BIN_To_HEX(S)

    #if we multiply by "02" we should check the first byte, if it's "0" we will just shift it to the left, if it's "1" we will shift it and do XOR operation with "1b"
    elif(S[0] == '0' and M == '02'):  
        result = BIN_To_HEX(S[1:len(S)]+'0')
    elif(S[0] == '1' and M == '02'):
        result = HEX_XOR(BIN_To_HEX(S[1:len(S)]+'0') , '1b')

    #if we multiply by "03" then we will do multiplication with "02" and then "01"    
    elif(M == '03'):
        result = HEX_XOR(GF_Mul(BIN_To_HEX(S) , '02') , BIN_To_HEX(S))

    #if we multiply by "09" then we will do multiplication with "02" three times and then "01" to become "09"
    elif(M == '09'):
        step1 = GF_Mul(BIN_To_HEX(S) , '02')
        step2 = GF_Mul(step1 , '02')
        step3 = GF_Mul(step2 , '02')
        result = HEX_XOR(step3 , BIN_To_HEX(S))

    #if we multiply by "0b" then we will do multiplication with "02" two times and then "01" then "02" then "01" to become "0b"    (b stands for 11 in decimal)
    elif(M == '0b'):
        step1 = GF_Mul(BIN_To_HEX(S) , '02')
        step2 = GF_Mul(step1 , '02')
        step3 = HEX_XOR(step2 , BIN_To_HEX(S))
        step4 = GF_Mul(step3 , '02')
        result = HEX_XOR(step4 , BIN_To_HEX(S))

    #if we multiply by "0d" then we will do multiplication with "02" and then "01" then "02" two times then "01" to become "0d"    (d stands for 13 in decimal)    
    elif(M == '0d'):
        step1 = GF_Mul(BIN_To_HEX(S) , '02')
        step2 = HEX_XOR(step1 , BIN_To_HEX(S))
        step3 = GF_Mul(step2 , '02')
        step4 = GF_Mul(step3 , '02')
        result = HEX_XOR(step4 , BIN_To_HEX(S))

    #if we multiply by "0e" then we will do multiplication with "02" and then "01" then "02" then "01" then "02" to become "0e"    (e stands for 14 in decimal)     
    elif(M == '0e'):
        step1 = GF_Mul(BIN_To_HEX(S) , '02')
        step2 = HEX_XOR(step1 , BIN_To_HEX(S))
        step3 = GF_Mul(step2 , '02')
        step4 = HEX_XOR(step3 , BIN_To_HEX(S))
        result = GF_Mul(step4 , '02')
        
    return result


# In[14]:


#the shift rows function shift the four rows of a given input matrix
#takes input matrix and mode (E or D)
def Shift_Rows(state_matrix, mode):
  R,C = state_matrix.shape
  #iterate of the last three rows and shift them according to the mode
  for i in range(1,R):
    if mode == 'E':
      state_matrix[i,:] = rotate(list(state_matrix[i,:]), i)
    else:
      state_matrix[i,:] = rotate(list(state_matrix[i,:]), -i)
  return state_matrix


# In[15]:


#function to substitue hexa number with its number in the subsitution box in the table
#input matrix
#output subitituted matrix 

def Sub_Byte(state_matrix , mode):
    if(mode == 'E'):
        sbox = [
            [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
            [0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
            [0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
            [0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
            [0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
            [0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
            [0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
            [0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
            [0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
            [0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
            [0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
            [0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
            [0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
            [0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
            [0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
            [0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]
            ]
    elif(mode == 'D'):
        sbox = [
            [0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB],
            [0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB],
            [0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E],
            [0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25],
            [0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92],
            [0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84],
            [0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06],
            [0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B],
            [0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73],
            [0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E],
            [0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B],
            [0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4],
            [0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F],
            [0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF],
            [0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61],
            [0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D]
            ]
    
    s_state_matrix = ''
    if(type(state_matrix) is not str):
        R,C = 4 , 4
        s_state_matrix = np.zeros((R,C) , dtype = np.chararray)
        for i in range(R):
            for j in range(C):
                position = state_matrix[i][j]

                hex_element = str(hex(sbox[int(position[0],16)][int(position[1],16)])).replace('0x' , '')  #sbox gets positions in the horizental and vertical axis
                if(len(hex_element) == 1):
                    hex_element = '0' + hex_element
                s_state_matrix[i][j] = hex_element ##output matrix
        

    else:       

        for i in range(0 , len(state_matrix)-1 , 2):
            hex_element = str(hex(sbox[int(state_matrix[i],16)][int(state_matrix[i+1],16)])).replace('0x' , '') #sbox gets positions in the horizental and vertical axis
            if(len(hex_element) == 1):
                hex_element = '0' + hex_element
            s_state_matrix += hex_element #output string

    return s_state_matrix


# In[16]:


#function for add round key layer in AES
#get input Matrix and key 
#output matrix
def Add_Round_Key(input_matrix , key):
    R,C = 4,4
    output_matrix = np.zeros((R,C) , dtype = np.chararray)

    for j in range(C):
        output_column = HEX_Column_To_String(input_matrix[:,j]) #convert hexadecimal column to string
        result_column = add_zero(HEX_XOR(output_column , key[j])) #Xor state column with key column
        position = 0
        for i in range(R):
            output_matrix[i][j] = result_column[position:position+2] #reconstruct the matrix
            position += 2
      
    return output_matrix     


# In[17]:


#function to do the mix cloumns layer in AES using two operations Galois field multiplication and the XOR operation
#gets input matrix and mode (E or D)
#output updated mixed matrix

def Mix_Columns(state_matrix , mode):
    R,C = state_matrix.shape
    #check if mode is E or D
    if(mode == 'E'):
        matrix = [['02','03','01','01'],
            ['01','02','03','01'],
            ['01','01','02','03'],
            ['03','01','01','02']]
        
    elif(mode == 'D'):
        matrix = [['0e','0b','0d','09'],
            ['09','0e','0b','0d'],
            ['0d','09','0e','0b'],
            ['0b','0d','09','0e']]
    
    output_mix_matrix = np.zeros((R,C) , dtype = np.chararray)
    result = []
    
    #iterate over the columns and the raws
    for i in range(R):
        for j in range(C):
            xor_result = '00'
            for k in range(R):
                result = GF_Mul(state_matrix[k][j] , matrix[i][k]) #Do the Galois field multiplication
                xor_result = HEX_XOR(xor_result , result) #XOR the result to sum the result of all elements 
                        
            output_mix_matrix[i][j] = xor_result
            
    return output_mix_matrix


'''
Function for AES-256 Encryption & Decryption
Parameters:
    plain_text
    key
    mode (E for Encryption, D for Decryption)
Returns:
    cipher_text or plain_text
'''
def AES_256_ECB(message , key , mode):
    #Convert message to matrix
    state_matrix = Convert_to_Matrix(message)
    #do key expansion
    expanded_key = Key_Expand_256(key)
 
    #check if mode is E
    if(mode == 'E'):
        #First thing to do is Add_Round_Key layer
        round_input = Add_Round_Key(state_matrix , expanded_key[0:4]) #intial round key
        position = 4
        #iterate over the four layers for 14 rounds
        for i in range(14):
            #substituion layer
            step1 = Sub_Byte(round_input , mode) 
            #shift rows layer
            step2 = Shift_Rows(step1 , mode)   
            #at the final round, mix columns is skipped
            if(i != 13):
                #mix columns layer
                step3 = Mix_Columns(step2 , mode) 
                #add round key layer
                step4 = Add_Round_Key(step3 , expanded_key[position:position+4]) 
                position += 4
                round_input = step4
            else:
                #final add round key
                cipher_text = Add_Round_Key(step2 , expanded_key[position:position+4]) 
        return cipher_text
    
    #if mode is D, do decryption
    elif(mode == 'D'):
        #do Add_Round_Key layer
        round_input = Add_Round_Key(state_matrix , expanded_key[56:60])
        position = 52
        #iterate over the four layers for 14 rounds
        for i in range(14):
            #shift rows layer
            step1 = Shift_Rows(round_input , mode) 
            #substituion layer
            step2 = Sub_Byte(step1 , mode)  
            #at the final round, mix columns is skipped
            if(i != 13):
                #add round key layer
                step3 = Add_Round_Key(step2 , expanded_key[position:position+4]) 
                #mix columns layer
                step4 = Mix_Columns(step3 , mode) 
                position -= 4
                round_input = step4
            else:
                #final round key layer
                plain_text = Add_Round_Key(step2 , expanded_key[position:position+4]) 
        return plain_text

In [2]:
#function to make the XOR between Hexadecimal numbers
#takes two hexa numbers and output the result
def HEX_XOR(A , B):
    x =  str(A) #add 0x before every input
    y =  str(B)
    
    q = int(x , 16)
    p = int(y , 16)

    result = hex(q ^ p) #do the XOR
    result = result.replace('0x' , '')
    #check if there the result is only one element, add zero
    if(len(result) < len(A)):
        for i in range(len(A)-len(result)):
            result = '0'+ result
                
    return result

In [3]:
'''
Helper function to split input string into blocks
Parameters:
  input string
Returns:
  blocks
'''
def get_string_blocks(stringg):
  #get length of string
  length = len(stringg)
  #get number of blocks
  blocks_number = length / 32
  #initialize empty list to hold the blocks
  blocks = []
  #iterate over the number of blocks
  for i in range(int(blocks_number)):
    #divide the string into blocks of size 32
    blocks.append(stringg[32*i: 32*(i+1)] )
  #return the blocks
  return blocks


In [4]:
'''
Function Name: Matrix_To_String
  Convert input matrix to string
Parameters:
  input matrix
Returns:
  String containing the elements of the input matrix
'''
def Matrix_To_String(matrix):
    #gets size of matrix
    R = len(matrix)
    C = len(matrix[0])

    #iniztilize empty string
    string = '' 
    
    #iterate over the rows and the coloumns
    for j in range(C):
        for i in range(R):
            #add every element of the matrix to the string
            string += matrix[i][j] 
    
    #return string
    return string

## CTR Mode

In [5]:
pip install pycrypto

In [6]:
from datetime import datetime
from Crypto.Cipher import AES

#### CTR 256 Parallized

In [7]:
def encrypt_block_CTR(plaintext, key, iv):
    """
     function to implement block AES-256 for encryption for CTR Mode.
     Parameters: plaintext, key, iv (initialization vector)
     Returns: encrypted message
    """
    
    plaintext_len = len(plaintext)
    ## Zero padding for the text to be multiples of 128
    remainder = plaintext_len % 32
    if remainder != 0:
      width = plaintext_len + (32 - remainder)
      plaintext = plaintext.ljust(width,"0")

    # Divide the plaintext into blocks
    blocks = get_string_blocks(plaintext)

    # Intialize counter with iv 
    counter = []
    for i in range(len(blocks)):
      # Do encryption for every counter block using the function AES_256_ECB
      #encryptor = AES.new(key, AES.MODE_ECB)
      result = AES_256_ECB(iv,key,mode="E")
      #result = encryptor.encrypt(iv)
      counter.append(Matrix_To_String(result))
      #counter.append(result.hex())
      # Increment the counter by 1
      c = int(iv, base = 16)
      iv = hex(c+1)

    encrypted_blocks = []
    # Iterate over every block
    for i, block in enumerate(blocks):
        #do XOR between the hexadicimal blcok and initialization vector
        xor_result = HEX_XOR(block,counter[i])
        #append the result to the list encrypted_blocks
        encrypted_blocks.append(xor_result)

    cipher = "".join(encrypted_blocks)
    #return the encrypted_blocks
    return cipher[: plaintext_len]


In [8]:
def decrypt_block_CTR(ciphertext, key, iv):
    """
     function to implement block AES-256 for decryption for CTR Mode.
     Parameters: ciphertext, key, iv (initialization vector)
     Returns: decrypted message
    """
    ciphertext_len = len(ciphertext)
    ## Zero padding for the text to be multiples of 128
    remainder = ciphertext_len % 32
    if remainder != 0:
      width = ciphertext_len + (32 - remainder)
      ciphertext = ciphertext.ljust(width,"0")
    
    # Divide the ciphertext into blocks
    blocks = get_string_blocks(ciphertext)
    
    # Intialize counter with iv 
    counter = []
    for i in range(len(blocks)):
      # Do encryption for every counter block using the function AES_256_ECB
      result = AES_256_ECB(iv,key,mode="E")
      counter.append(Matrix_To_String(result))
      # Increment the counter by 1
      c = int(iv, base = 16)
      iv = hex(c+1)

    decrypted_blocks = []
    # Iterate over every block
    for i, block in enumerate(blocks):
        #do XOR between the hexadicimal blcok and initialization vector
        xor_result = HEX_XOR(block,counter[i])
        #append the result to the list decrypted_blocks
        decrypted_blocks.append(xor_result)

    plaintext = "".join(decrypted_blocks)
    #return the decrypted_blocks
    return plaintext[: ciphertext_len]


#### CTR 256 Ordinary

In [9]:
def encrypt_block_CTR_ordinary(plaintext, key, iv):
    """
     function to implement block AES-256 for encryption for CTR Mode.
     Parameters: plaintext, key, iv (initialization vector)
     Returns: encrypted message
    """
    
    plaintext_len = len(plaintext)
    ## Zero padding for the text to be multiples of 128
    remainder = plaintext_len % 32
    if remainder != 0:
      width = plaintext_len + (32 - remainder)
      plaintext = plaintext.ljust(width,"0")

    # Divide the plaintext into blocks
    blocks = get_string_blocks(plaintext)

    encrypted_blocks = []
    # Iterate over every block
    for i, block in enumerate(blocks):
        # Do encryption for every counter block using the function AES_256_ECB
        result = AES_256_ECB(iv,key,mode="E")
        counter = Matrix_To_String(result)
        #do XOR between the hexadicimal blcok and initialization vector
        xor_result = HEX_XOR(block,counter)
        #append the result to the list encrypted_blocks
        encrypted_blocks.append(xor_result)
        # Increment the counter by 1
        c = int(iv, base = 16)
        iv = hex(c+1)

    cipher = "".join(encrypted_blocks)
    #return the encrypted_blocks
    return cipher[: plaintext_len]


In [10]:
def decrypt_block_CTR_ordinary(ciphertext, key, iv):
    """
     function to implement block AES-256 for decryption for CTR Mode.
     Parameters: ciphertext, key, iv (initialization vector)
     Returns: decrypted message
    """
    ciphertext_len = len(ciphertext)
    ## Zero padding for the text to be multiples of 128
    remainder = ciphertext_len % 32
    if remainder != 0:
      width = ciphertext_len + (32 - remainder)
      ciphertext = ciphertext.ljust(width,"0")
    
    # Divide the ciphertext into blocks
    blocks = get_string_blocks(ciphertext)
    
    decrypted_blocks = []
    # Iterate over every block
    for i, block in enumerate(blocks):
        # Do encryption for every counter block using the function AES_256_ECB
        result = AES_256_ECB(iv,key,mode="E")
        counter = Matrix_To_String(result)
        #do XOR between the hexadicimal blcok and initialization vector
        xor_result = HEX_XOR(block,counter)
        #append the result to the list decrypted_blocks
        decrypted_blocks.append(xor_result)
        # Increment the counter by 1
        c = int(iv, base = 16)
        iv = hex(c+1)

    plaintext = "".join(decrypted_blocks)
    #return the decrypted_blocks
    return plaintext[: ciphertext_len]


#### CTR 128

In [11]:
def encrypt_block_CTR_128(plaintext, key, iv):
    """
     function to implement block AES-128 for encryption for CTR Mode.
     Parameters: plaintext, key, iv (initialization vector)
     Returns: encrypted message
    """
    
    plaintext_len = len(plaintext)
    ## Zero padding for the text to be multiples of 128
    remainder = plaintext_len % 32
    if remainder != 0:
      width = plaintext_len + (32 - remainder)
      plaintext = plaintext.ljust(width,"0")

    # Divide the plaintext into blocks
    blocks = get_string_blocks(plaintext)

    # Intialize counter with iv 
    counter = []
    for i in range(len(blocks)):
      iv_len = len(iv)
      # Do encryption for every counter block using the function AES_128
      # Convert the key  from hex to bytes
      key_byte = bytes.fromhex(key)
      encryptor = AES.new(key_byte, AES.MODE_ECB)
      #convert the counter from hex to bytes
      iv_bytes = bytes.fromhex(iv)
      # Encrypt the counter
      result = encryptor.encrypt(iv_bytes)
      counter.append(result.hex())
      # Increment the counter by 1
      c = int(iv, base = 16)
      iv = hex(c+1)
      iv = iv.replace('0x' , '')
      #check if there the result is only one element, add zero
      if(len(iv) < iv_len):
        for i in range(iv_len-len(iv)):
           iv = '0'+ iv
                

    encrypted_blocks = []
    # Iterate over every block
    for i, block in enumerate(blocks):
        #do XOR between the hexadicimal blcok and initialization vector
        xor_result = HEX_XOR(block,counter[i])
        #append the result to the list encrypted_blocks
        encrypted_blocks.append(xor_result)

    cipher = "".join(encrypted_blocks)
    #return the encrypted_blocks
    return cipher[: plaintext_len]


In [12]:
def decrypt_block_CTR_128(ciphertext, key, iv):
    """
     function to implement block AES-256 for decryption for CTR Mode.
     Parameters: ciphertext, key, iv (initialization vector)
     Returns: decrypted message
    """
    ciphertext_len = len(ciphertext)
    ## Zero padding for the text to be multiples of 128
    remainder = ciphertext_len % 32
    if remainder != 0:
      width = ciphertext_len + (32 - remainder)
      ciphertext = ciphertext.ljust(width,"0")
    
    # Divide the ciphertext into blocks
    blocks = get_string_blocks(ciphertext)
    
    # Intialize counter with iv 
    counter = []
    for i in range(len(blocks)):
      iv_len = len(iv)
      # Do encryption for every counter block using the function AES_128
       # Convert the key  from hex to bytes
      key_byte = bytes.fromhex(key)
      encryptor = AES.new(key_byte, AES.MODE_ECB)
      #convert the counter from hex to bytes
      iv_bytes = bytes.fromhex(iv)
      # Encrypt the counter
      result = encryptor.encrypt(iv_bytes)
      counter.append(result.hex())
      # Increment the counter by 1
      c = int(iv, base = 16)
      iv = hex(c+1)
      iv = iv.replace('0x' , '')
      #check if there the result is only one element, add zero
      if(len(iv) < iv_len):
        for i in range(iv_len-len(iv)):
           iv = '0'+ iv
                

    decrypted_blocks = []
    # Iterate over every block
    for i, block in enumerate(blocks):
        #do XOR between the hexadicimal blcok and initialization vector
        xor_result = HEX_XOR(block,counter[i])
        #append the result to the list decrypted_blocks
        decrypted_blocks.append(xor_result)

    plaintext = "".join(decrypted_blocks)
    #return the decrypted_blocks
    return plaintext[: ciphertext_len]


### Testing AES-128-CTR

##### Single block

In [13]:
start_time = datetime.now()
output = encrypt_block_CTR_128("53696E676C6520626C6F636B206D7367","AE6852F8121067CC4BF7A5765577F39E", "00000030000000000000000000000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  e4095d4fb7a7b3792d6175a3261311b8
time total: 0:00:00.000205


In [14]:
start_time = datetime.now()
output = decrypt_block_CTR_128("e4095d4fb7a7b3792d6175a3261311b8","AE6852F8121067CC4BF7A5765577F39E", "00000030000000000000000000000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  53696e676c6520626c6f636b206d7367
time total: 0:00:00.000221


##### Multiple blocks

In [15]:
start_time = datetime.now()
output = encrypt_block_CTR_128("000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F20212223","7691BE035E5020A8AC6E618529F9A0DC", "00E0017B27777F3F4A1786F000000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  c1cf48a89f2ffdd9cf4652e9efdb72d74540a42bde6d7836d59a5ceaaef3105325b2072f
time total: 0:00:00.001065


In [16]:
start_time = datetime.now()
output = decrypt_block_CTR_128("c1cf48a89f2ffdd9cf4652e9efdb72d74540a42bde6d7836d59a5ceaaef3105325b2072f","7691BE035E5020A8AC6E618529F9A0DC", "00E0017B27777F3F4A1786F000000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f20212223
time total: 0:00:00.000221


### Testing AES-256-CTR 

##### Single block Parallized

In [17]:
Key= "F6D66D6BD52D59BB0796365879EFF886C66DD51A5B6A99744B50590C87A23884"
ctr1 = "00FAAC24C1585EF15A43D87500000001"
plaintext ="000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F"
#ciphertext:F05E231B3894612C49EE000B804EB2A9B8306B508F839D6A5530831D9344AF1C"

In [18]:
start_time = datetime.now()
output = encrypt_block_CTR(plaintext,Key, ctr1)
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  f05e231b3894612c49ee000b804eb2a9b8306b508f839d6a5530831d9344af1c
time total: 0:00:00.021854


In [19]:
start_time = datetime.now()
output = encrypt_block_CTR("53696E676C6520626C6F636B206D7367","776BEFF2851DB06F4C8A0542C8696F6C6A81AF1EEC96B4D37FC1D689E6C1C104", "00000060DB5672C97AA8F0B200000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  145ad01dbf824ec7560863dc71e3e0c0
time total: 0:00:00.010325


In [20]:
start_time = datetime.now()
output = decrypt_block_CTR("145ad01dbf824ec7560863dc71e3e0c0","776BEFF2851DB06F4C8A0542C8696F6C6A81AF1EEC96B4D37FC1D689E6C1C104", "00000060DB5672C97AA8F0B200000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  53696e676c6520626c6f636b206d7367
time total: 0:00:00.014672


##### Single block Ordinary

In [21]:
start_time = datetime.now()
output = encrypt_block_CTR_ordinary("53696E676C6520626C6F636B206D7367","776BEFF2851DB06F4C8A0542C8696F6C6A81AF1EEC96B4D37FC1D689E6C1C104", "00000060DB5672C97AA8F0B200000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  145ad01dbf824ec7560863dc71e3e0c0
time total: 0:00:00.009712


In [22]:
start_time = datetime.now()
output = decrypt_block_CTR_ordinary("145ad01dbf824ec7560863dc71e3e0c0","776BEFF2851DB06F4C8A0542C8696F6C6A81AF1EEC96B4D37FC1D689E6C1C104", "00000060DB5672C97AA8F0B200000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  53696e676c6520626c6f636b206d7367
time total: 0:00:00.010313


##### Multiple blocks Parallized

In [23]:
start_time = datetime.now()
output = encrypt_block_CTR("000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F20212223","FF7A617CE69148E4F1726E2F43581DE2AA62D9F805532EDFF1EED687FB54153D", "001CC5B751A51D70A1C1114800000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  eb6c52821d0bbbf7ce7594462aca4faab407df866569fd07f48cc0b583d6071f1ec0e6b8
time total: 0:00:00.027552


In [24]:
start_time = datetime.now()
output = decrypt_block_CTR("eb6c52821d0bbbf7ce7594462aca4faab407df866569fd07f48cc0b583d6071f1ec0e6b8","FF7A617CE69148E4F1726E2F43581DE2AA62D9F805532EDFF1EED687FB54153D", "001CC5B751A51D70A1C1114800000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f20212223
time total: 0:00:00.018911


##### Multiple blocks ordinary

In [25]:
start_time = datetime.now()
output = encrypt_block_CTR_ordinary("000102030405060708090A0B0C0D0E0F101112131415161718191A1B1C1D1E1F20212223","FF7A617CE69148E4F1726E2F43581DE2AA62D9F805532EDFF1EED687FB54153D", "001CC5B751A51D70A1C1114800000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  eb6c52821d0bbbf7ce7594462aca4faab407df866569fd07f48cc0b583d6071f1ec0e6b8
time total: 0:00:00.023330


In [26]:
start_time = datetime.now()
output = decrypt_block_CTR_ordinary("eb6c52821d0bbbf7ce7594462aca4faab407df866569fd07f48cc0b583d6071f1ec0e6b8","FF7A617CE69148E4F1726E2F43581DE2AA62D9F805532EDFF1EED687FB54153D", "001CC5B751A51D70A1C1114800000001")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  000102030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f20212223
time total: 0:00:00.024037


### GCTR: it is just CTR-128 but with j0 as iv, and it is only on one block

### GHASH - and its helping functions

In [27]:
def Hex_to_Binary(hexnum, numbits):
  binarySeq = str(bin(int(hexnum, 16))).replace("0b", "").zfill(numbits)
  binarySeq = [int(char) for char in binarySeq]
  binarySeq = binarySeq[::-1]
  return binarySeq

In [28]:
def Binary_to_Hex(binaryseq, numofbits):
  rx = [int(abs(num)) for num in binaryseq]
  rx = rx + [0]*(numofbits-len(rx))
  rx = "".join([str(num) for num in rx])
  rx = rx[::-1]
  rx = hex(int(rx, 2)).replace("0x", "")
  return rx

In [29]:
def multiply_poly(p1, p2):
  res = [0]*(len(p1)+len(p2)-1)
  for power1,coeff1 in enumerate(p1):
      for power2,coeff2 in enumerate(p2):
          res[power1+power2] = (res[power1+power2]+(coeff1*coeff2))%2
  return res

In [30]:
def reduce_polynomial(num, den):
    # normalize the numerator and denumerator
    while num and num[-1] == 0:
        num.pop()
    if num == []:
        num.append(0)
    while den and den[-1] == 0:
        den.pop()
    if den == []:
        den.append(0)

    if len(num) >= len(den):
        #Shift denumerator towards right to be as num
        shiftlen = len(num) - len(den)
        den = [0] * shiftlen + den
    else:
        return [0], num

    quot = []
    divisor = den[-1]
    for i in range(shiftlen + 1):
        # Get the the quotient's coeff.
        mult = num[-1] / divisor
        quot = [int(mult)] + quot

        # xor (mult * den) from num in mod 2
        if mult != 0:
            d = [mult * u for u in den]
            num = [int((u + v)%2) for u, v in zip(num, d)]

        num.pop()
        den.pop(0)

    while num and num[-1] == 0:
        num.pop()
    if num == []:
        num.append(0)
    return num

In [31]:
def multiply_and_reduce(hex1, hex2, px, numofbits):
  a1 = Hex_to_Binary(hex1, numofbits)
  a2 = Hex_to_Binary(hex2, numofbits)
  ax = multiply_poly(a1, a2)
  # print(Binary_to_Hex(ax, numofbits))
  px = Hex_to_Binary(px, len(px)*4)
  rx = reduce_polynomial(ax, px)
  rx = Binary_to_Hex(rx, numofbits)
  return rx

In [32]:
# just testing a multiplication
multiply_and_reduce('0388DACE60B6A392F328C2B971B2FE78', '66E94BD4EF8A2C3B884CFA59CA342B2E', "100000000000000000000000000000087", 128)

'519fa38ac731568e9c1eb21731167f1c'

In [33]:
# The irreducible polynomial in GF(128)
px_GF128 = 10**128 +10**7 +10**2 +10+1
hex_px_GF128 = "100000000000000000000000000000087"

The GHASH that uses all of the above

In [34]:
def GHASH(hashKey, stringX):
    # Divide the string into blocks of 128
    blocks = get_string_blocks(stringX)

    # Get the y1 that will be Xored with the next block X2
    hex_px_GF128 = "100000000000000000000000000000087"
    y = multiply_and_reduce(blocks[0], hashKey, hex_px_GF128, 128)
    
    for i in range(1, len(blocks)):
      # Do XOR between the hexadicimal block and the previous y
      xor_result = HEX_XOR(blocks[i], y)
      # Multipling the xor_result with H
      y = multiply_and_reduce(xor_result, hashKey, hex_px_GF128, 128)

    return y

## GCM - Encryption

In [35]:
def GCM_encrypt(plaintext, key, iv, associatedData, hashKey):
  """
  this function takes:
      - plaintext
      - key
      - iv
      - associated data
      - hashkey
  and encrypts the plaintext, and append the associated data with their lenghts and produce a tag for all of that
  """
  # prepare the first counter from iv->j0->c1
  j0 = iv + "0"*7 + "1"
  c1 = iv + "0"*7 + "2"
  # Encrypt the plaintext with CTR function
  ciphertxt = encrypt_block_CTR_128(plaintext, key, c1)

  # Padding for the ciphertext
  ciphertxt_len = len(ciphertxt)
  remainder = ciphertxt_len % 32
  if remainder != 0:
    width = ciphertxt_len + (32 - remainder)
    ciphertxt = ciphertxt.ljust(width,"0")

  # padding for the associated data 
  associatedData_len = len(associatedData)
  remainder = associatedData_len % 32
  if remainder != 0:
    width = associatedData_len + (32 - remainder)
    associatedData = associatedData.ljust(width,"0")

  # calculating the length of associated data bits in hexa
  associatedData_hex_len = hex(associatedData_len*4) # since each hex char is 4 bits
  associatedData_hex_len = associatedData_hex_len.replace('0x' , '')
  # make sure size is 64 bit
  if(len(associatedData_hex_len) < 16):
    for i in range(16-len(associatedData_hex_len)):
        associatedData_hex_len = '0'+ associatedData_hex_len

  # calculating the length of ciphertxt bits in hexa  
  ciphertxt_hex_len = hex(ciphertxt_len*4) # since each hex char is 4 bits
  ciphertxt_hex_len = ciphertxt_hex_len.replace('0x' , '')
  # make sure size is 64 bit
  if(len(ciphertxt_hex_len) < 16):
    for i in range(16-len(ciphertxt_hex_len)):
        ciphertxt_hex_len = '0'+ ciphertxt_hex_len

  # Joining the four blocks togther
  stringX = associatedData + ciphertxt + associatedData_hex_len + ciphertxt_hex_len

  # GHash 
  ghash = GHASH(hashKey, stringX)

  # GCTR
  tag = encrypt_block_CTR_128(ghash, key, j0)
  return stringX,tag

In [36]:
plaintext = "08000F101112131415161718191A1B1C1D1E1F202122232425262728292A2B2C2D2E2F303132333435363738393A0002"
key = "AD7A2BD03EAC835A6F620FDCB506B345"
iv = "12153524C0895E81B2C28465"
associatedData = "D609B1F056637A0D46DF998D88E52E00B2C2846512153524C0895E81"
cipher = "701AFA1CC039C0D765128A665DAB69243899BF7318CCDC81C9931DA17FBE8EDD7D17CB8B4C26FC81E3284F2B7FBA713D"
hashKey = "73A23D80121DE2D5A850253FCF43120E"

# expected_ghash = "A4C350FB66B8C960E83363381BA90F50"


In [37]:
msg,tag = GCM_encrypt(plaintext, key, iv, associatedData, hashKey)
print(msg)
print(tag)

D609B1F056637A0D46DF998D88E52E00B2C2846512153524C0895E8100000000701afa1cc039c0d765128a665dab69243899bf7318ccdc81c9931da17fbe8edd7d17cb8b4c26fc81e3284f2b7fba713d00000000000000e00000000000000180
5e6c61d4e5795092bfdf9eab8b2647db


In [38]:
msg_1 = "D609B1F056637A0D46DF998D88E52E00B2C2846512153524C0895E8100000000701afa1cc039c0d765128a665dab69243899bf7318ccdc81c9931da17fbe8edd7d17cb8c4c26fc81e3284f2b7fba713d00000000000000e00000000000000180"

### GCM - Decryption

In [39]:
def GCM_decrypt(msg, tag, key, iv, hashKey):
  """
  this function takes:
      - msg
      - tag
      - key
      - iv
      - hashkey
  and extracts the ciphertxt and decrypts it, also checks if the tag is correct
  """
  # prepare the first counter from iv->j0->c1
  j0 = iv + "0"*7 + "1"
  c1 = iv + "0"*7 + "2"

  # extract the lengths of associated data and ciphertext
  cipher_len = int(msg[-16:], 16)//4 # in terms of hex chars 
  assoc_len = int(msg[-32:-16], 16)//4 # in terms of hex chars 

  # extract associated data
  if assoc_len % 32 == 0:
      assoc_actual_len = assoc_len
  else: 
      assoc_actual_len = assoc_len+(32 - (assoc_len % 32 ))
  associated_data = msg[0:assoc_len]

  # extract cipher text data
  if cipher_len % 32 == 0:
      cipher_actual_len = cipher_len
  else: 
      cipher_actual_len = cipher_len+(32 - (cipher_len % 32 ))
  ciphertxt = msg[assoc_actual_len:assoc_actual_len+cipher_actual_len]

  # Decrypt the ciphertxt with CTR function
  plaintext = encrypt_block_CTR_128(ciphertxt, key, c1)

  # try to reproduce the tag
  # # GHash 
  ghash = GHASH(hashKey, msg)
  # GCTR
  recomputed_tag = encrypt_block_CTR_128(ghash, key, j0)
  if tag == recomputed_tag:
    print("Tag verified")
  else:
    print("Tag isn't the same!!!")
  return associated_data, plaintext[0:cipher_len], recomputed_tag

In [40]:

associated_data, plaintext, recomputed_tag = GCM_decrypt(msg_1, tag, key, iv, hashKey)
print(associated_data)
print(plaintext)
print(recomputed_tag)

Tag isn't the same!!!
D609B1F056637A0D46DF998D88E52E00B2C2846512153524C0895E81
08000f101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f373132333435363738393a0002
13fbc597d5d8fe9cba164f6121cfc63f


In [41]:
plaintext = "08000F101112131415161718191A1B1C1D1E1F202122232425262728292A2B2C2D2E2F303132333435363738393A0002"
key = "AD7A2BD03EAC835A6F620FDCB506B345"
iv = "12153524C0895E81B2C28465"
associatedData = "D609B1F056637A0D46DF998D88E52E00B2C2846512153524C0895E81"
cipher = "701AFA1CC039C0D765128A665DAB69243899BF7318CCDC81C9931DA17FBE8EDD7D17CB8B4C26FC81E3284F2B7FBA713D"
hashKey = "73A23D80121DE2D5A850253FCF43120E"

# expected_ghash = "A4C350FB66B8C960E83363381BA90F50"

### GCM Encryption time tests

test on given test case

In [42]:
start_time = datetime.now()
msg,tag = GCM_encrypt(plaintext, key, iv, associatedData, hashKey)
end_time = datetime.now()
total_time = end_time - start_time
print('time total: {}'.format(total_time))

time total: 0:00:00.058636


In [43]:
start_time = datetime.now()
associated_data, plaintext, recomputed_tag = GCM_decrypt(msg, tag, key, iv, hashKey)
end_time = datetime.now()
total_time = end_time - start_time
print('time total: {}'.format(total_time))

Tag verified
time total: 0:00:00.059954


## Other modes

In [44]:
#ECB MODE:

"""
function to implement block AES-256 for encryption ECB Mode.
Parameters:
  plaintext 
  key
Returns:
  encrypted message
"""
def encrypt_block_ECB(plaintext,key):
    #divide the plaintext into blocks
    blocks = get_string_blocks(plaintext)
    #initialize empty list to hold the encrypted blocks
    encrypted_blocks = []
    #iterate over every block and do the encryption process by calling the
    #function AES_256_ECB that takes input block, key and mode E
    for block in blocks:
        #do encryption for every block
        result = AES_256_ECB(block,key,mode="E")
        #append the result to the list encrypted_blocks
        encrypted_blocks.append(Matrix_To_String(result))
    #return the encrypted_blocks
    return "".join(encrypted_blocks)


"""
function to implement block AES-256 for decryption ECB Mode.
Parameters:
  ciphertext 
  key
Returns:
  decrypted blocks
"""
def decrypt_block_ECB(ciphertext,key):
    #divide the ciphertext intto blocks
    blocks = get_string_blocks(ciphertext)
    #initialize empty list to hold the decrypted blocks
    decrypted_blocks = []
    #iterate over every block and do the decryption process by calling the
    #function AES_256_ECB that takes input block, key and mode D
    for block in blocks:
        #do decryption for every block
        result = AES_256_ECB(block,key,mode="D")
        #append the result to the list decrypted_blocks
        decrypted_blocks.append(Matrix_To_String(result))
    #return the decrypted_blocks
    return "".join(decrypted_blocks)

In [45]:
# CBC MODE:

"""
function to implement block AES-256 for encryption for CBC Mode.
Parameters:
  plaintext 
  key
  iv (initialization vector)
Returns:
  encrypted message
"""
def encrypt_block_CBC(plaintext,key,iv):
    #divide the plaintext into blocks
    blocks = get_string_blocks(plaintext)
    #initialize empty list to hold the encrypted blocks
    encrypted_blocks = []
    #iterate over every block
    for block in blocks:
        #do XOR between the hexadicimal blcok and initialization vector
        xor_result = HEX_XOR(block,iv)
        #do encryption for every block using the function AES_256_ECB
        result = AES_256_ECB(xor_result,key,mode="E")
        #append the result to the list encrypted_blocks
        encrypted_blocks.append(Matrix_To_String(result))
        #update iv to the previous value of the encrypted_blocks
        iv = encrypted_blocks[-1]
    #return the encrypted_blocks
    return "".join(encrypted_blocks)


"""
function to implement block AES-256 for decryption CBC Mode.
Parameters:
  ciphertext 
  key
  iv (initialization vector)
Returns:
  decrypted blocks
"""
def decrypt_block_CBC(ciphertext,key,iv):
    #divide the ciphertext into blocks
    blocks = get_string_blocks(ciphertext)
    #initialize empty list to hold the decrypted blocks
    decrypted_blocks = []
    blocks_buffer = []
    #iterate over every block
    for block in blocks:
        blocks_buffer.append(block)
        #do decryption for every block using the function AES_256_ECB
        result = AES_256_ECB(block,key,mode="D")
        #do XOR for the hexadecimal result and initialization vector
        xor_result = HEX_XOR(Matrix_To_String(result),iv)
        #append the xor_result to the decrypted_blocks
        decrypted_blocks.append(xor_result)
        #update iv to be equal the previous block of the ciphertext
        iv = blocks_buffer[-1]
    #return the decrypted_blocks
    return "".join(decrypted_blocks)

In [46]:
# CFB MODE (the one like openssl):

"""
function to implement block AES-256 for encryption for CFB Mode.
Parameters:
  plaintext 
  key
  iv (initialization vector)
Returns:
  encrypted message
"""
def encrypt_CFB(plaintext,key,iv):
    #divide the plaintext into blocks
    blocks = get_string_blocks(plaintext)
    #initialize empty list to hold the encrypted blocks
    encrypted_blocks = []
    #iterate over every block
    for block in blocks:
        #do encryption for the initialization vector and the key
        result = Matrix_To_String(AES_256_ECB(iv,key,mode='E'))
        #do XOR for the hexadecimal result and block
        xor_result = HEX_XOR(result, block)
        #append the xor_result to the encrypted_blocks
        encrypted_blocks.append(xor_result)
        #update the initialization vector to equal the xor_result
        iv = xor_result
    #return the encrypted_blocks
    return "".join(encrypted_blocks)


"""
function to implement block AES-256 for decryption CFB Mode.
Parameters:
  ciphertext 
  key
  iv (initialization vector)
Returns:
  decrypted blocks
"""
def decrypt_CFB(ciphertext, key, iv):
    #divide the ciphertext into blocks
    blocks = get_string_blocks(ciphertext)
    #initialize empty list to hold the decrypted blocks
    decrypted_blocks = []
    #iterate over every block   
    for block in blocks:
        #do encryption for the initialization vector and the key
        result = Matrix_To_String(AES_256_ECB(iv,key,mode='E'))
        #do XOR for the hexadecimal result and block
        xor_result = HEX_XOR(block, result)
        #append the xor_result to the encrypted_blocks
        decrypted_blocks.append(xor_result)
        #update the initialization vector to equal the block
        iv = block
    return "".join(decrypted_blocks)

##Testing

Testing ECB:

In [47]:
start_time = datetime.now()
output = encrypt_block_ECB("91fbef2d15a97816060bee1feaa49afe","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  05834df1118469393e83d7c9c6ba2a7b
time total: 0:00:00.013488


In [48]:
start_time = datetime.now()
output = decrypt_block_ECB("05834df1118469393e83d7c9c6ba2a7b","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  91fbef2d15a97816060bee1feaa49afe
time total: 0:00:00.020168


Testing CBC:

In [49]:
start_time = datetime.now()
output = encrypt_block_CBC("91fbef2d15a97816060bee1feaa49afe","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566", "abc12345abc000abfde8789ae124bccd")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  24642c54f02225fcf6cd23d6064d63e3
time total: 0:00:00.010065


In [50]:
start_time = datetime.now()
output = decrypt_block_CBC("24642c54f02225fcf6cd23d6064d63e3","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566", "abc12345abc000abfde8789ae124bccd")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  91fbef2d15a97816060bee1feaa49afe
time total: 0:00:00.028144


Testing CFB (like openssl):

In [51]:
start_time = datetime.now()
output = encrypt_CFB("91fbef2d15a97816060bee1feaa49afe","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566", "abc12345abc000abfde8789ae124bccd")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  75a21df06481ef3b3cee8c6384893109
time total: 0:00:00.008869


In [52]:
start_time = datetime.now()
output = decrypt_CFB("75a21df06481ef3b3cee8c6384893109","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566", "abc12345abc000abfde8789ae124bccd")
end_time = datetime.now()
total_time = end_time - start_time
print("The decrypted: " ,output)
print('time total: {}'.format(total_time))

The decrypted:  91fbef2d15a97816060bee1feaa49afe
time total: 0:00:00.009323


COMPARISON WITH LONGER PLAINTEXT

In [53]:
# ECB
start_time = datetime.now()
output = encrypt_block_ECB("cf52e5c3954c51b94c9e38acb8c9a7c76aebdaa9943eae0a1ce155a2efdb4d46985d935511471452d9ee64d2461cb2991d59fc0060697f9a671672163230f367fed1422316e52d29eceacb8768f56d9b80f6d278093c9a8acd3cfd7edd8ebd5c293859f64d2f8486ae1bd593c65bc014","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  3a12579604b1716efac2b7cbe7c258a5f9bbd5c3445dacb3556090c8a14c73b5dd63acb1c075f6b4b2ccbc5d3771ef889dec552c0f26a69ae7b80d65e5d4262e97ad2d7288815cd0b0b5091485629c2e1030b7ca83465d926c251fcda56af4dd63a342b1d206f6ed95ebbf3718f79ff3
time total: 0:00:00.049738


In [54]:
# CBC
start_time = datetime.now()
output = encrypt_block_CBC("cf52e5c3954c51b94c9e38acb8c9a7c76aebdaa9943eae0a1ce155a2efdb4d46985d935511471452d9ee64d2461cb2991d59fc0060697f9a671672163230f367fed1422316e52d29eceacb8768f56d9b80f6d278093c9a8acd3cfd7edd8ebd5c293859f64d2f8486ae1bd593c65bc014","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566", "abc12345abc000abfde8789ae124bccd")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  4646602778b1210b37d96e70e0ba56d3d41fc7f03e872a155f9dbf9d2117e818a08a09057ce185f4ed4be6d6d48af6863b6f8b94989ed6f7e1a3ccd3ccc9bca736182ad6b9207bfa375224af4e7bab59be5a91a90d71394d5f59a37ad10676efec22c3fbd8e83a0eb66bfc0d212b4aae
time total: 0:00:00.044943


In [55]:
# CFB
start_time = datetime.now()
output = encrypt_CFB("cf52e5c3954c51b94c9e38acb8c9a7c76aebdaa9943eae0a1ce155a2efdb4d46985d935511471452d9ee64d2461cb2991d59fc0060697f9a671672163230f367fed1422316e52d29eceacb8768f56d9b80f6d278093c9a8acd3cfd7edd8ebd5c293859f64d2f8486ae1bd593c65bc014","3c4baaf4bdfa1e3de57891cfde4522ab3c4b6fff21a3b5ed56e11a99a0cd4566", "abc12345abc000abfde8789ae124bccd")
end_time = datetime.now()
total_time = end_time - start_time
print("The ciphertext: " ,output)
print('time total: {}'.format(total_time))

The ciphertext:  2b0b171ee464c694767b5ad0d6e40c300ee95316f799b46ba6f1e6f631a63706912d2500fdea7104baea3c958fe3af8188823f94d22a0c3a228acbadffd39013100b7b80f7d6672045bf42505aaec66b7e98339cedd1df2d9d81683eea60a2cff8704a9975fb42cbf784a3015b08e3a6
time total: 0:00:00.045618
